In [ ]:
#Load libraries
import os
import numpy as np
import torch
import glob
import torch.nn as nn
from torchvision.transforms import transforms
from torch.utils.data import DataLoader
from torch.optim import Adam
from torch.autograd import Variable
from torch.optim.lr_scheduler import StepLR
import torchvision
import pathlib
from sklearn.metrics import confusion_matrix

In [ ]:
#checking for device
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print(device)

In [ ]:
#Transforms
transformer=transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),  #0-255 to 0-1, numpy to tensors
    transforms.Normalize([0.5,0.5,0.5], # 0-1 to [-1,1] , formula (x-mean)/std
                        [0.5,0.5,0.5])
])

In [ ]:
#Dataloader

#Path for training and testing directory
train_path='./resized_data_split/train'
test_path='./resized_data_split/test'

#set the batch size for CNN
train_loader_32=DataLoader(
    torchvision.datasets.ImageFolder(train_path,transform=transformer),
    batch_size=32, shuffle=True
)
test_loader_32=DataLoader(
    torchvision.datasets.ImageFolder(test_path,transform=transformer),
    batch_size=32, shuffle=True
)

train_loader_100=DataLoader(
    torchvision.datasets.ImageFolder(train_path,transform=transformer),
    batch_size=100, shuffle=True
)
test_loader_100=DataLoader(
    torchvision.datasets.ImageFolder(test_path,transform=transformer),
    batch_size=100, shuffle=True
)

In [ ]:
#categories
root=pathlib.Path(train_path)
classes=sorted([j.name.split('/')[-1] for j in root.iterdir()])

print(classes)

In [ ]:
class ConvNet(nn.Module):
    def __init__(self,num_classes=5):
        super(ConvNet,self).__init__()
        
        #Output size after convolution filter
        #((w-f+2P)/s) +1
        
        #Input shape= (256,3,150,150)
        
        self.conv1=nn.Conv2d(in_channels=3,out_channels=12,kernel_size=3,stride=1,padding=1)
        #Shape= (256,12,150,150)
        self.bn1=nn.BatchNorm2d(num_features=12)
        #Shape= (256,12,150,150)
        self.relu1=nn.ReLU()
        #Shape= (256,12,150,150)
        
        self.pool=nn.MaxPool2d(kernel_size=2)
        #Reduce the image size be factor 2
        #Shape= (256,12,75,75)
        
        self.conv2=nn.Conv2d(in_channels=12,out_channels=20,kernel_size=3,stride=1,padding=1)
        #Shape= (256,20,75,75)
        self.relu2=nn.ReLU()
        #Shape= (256,20,75,75)
        
        self.conv3=nn.Conv2d(in_channels=20,out_channels=32,kernel_size=3,stride=1,padding=1)
        #Shape= (256,32,75,75)
        self.bn3=nn.BatchNorm2d(num_features=32)
        #Shape= (256,32,75,75)
        self.relu3=nn.ReLU()
        #Shape= (256,32,75,75)
        
        self.fc=nn.Linear(in_features=75 * 75 * 32,out_features=num_classes)
        
        #Feed forwad function
        
    def forward(self,input):
        output=self.conv1(input)
        output=self.bn1(output)
        output=self.relu1(output)
            
        output=self.pool(output)
            
        output=self.conv2(output)
        output=self.relu2(output)
            
        output=self.conv3(output)
        output=self.bn3(output)
        output=self.relu3(output)
            
        #Above output will be in matrix form, with shape (256,32,75,75)
            
        output=output.view(-1,32*75*75)
            
        output=self.fc(output)
            
        return output

In [ ]:
num_epochs=8

In [ ]:
model=ConvNet(num_classes=5).to(device)
loss_function=nn.CrossEntropyLoss()

In [ ]:
#calculating the size of training and testing images
train_count=len(glob.glob(train_path+'/**/*.jpg'))
test_count=len(glob.glob(test_path+'/**/*.jpg'))

print(train_count,test_count)

In [ ]:
optimizer=Adam(model.parameters(),lr=0.001,weight_decay=0.0001)

model for no weights, no drop prob and size 32

In [ ]:
# Model training and saving the best model

# Initialize the best accuracy variable
best_accuracy = 0.0

# Iterate through each epoch
for epoch in range(num_epochs):
    
    # Set the model to training mode
    model.train()
    
    # Initialize training accuracy and loss
    train_accuracy = 0.0
    train_loss = 0.0
    
    # Iterate through the training dataset
    for i, (images, labels) in enumerate(train_loader_32):
        # Move images and labels to GPU if available
        if torch.cuda.is_available():
            images = Variable(images.cuda())
            labels = Variable(labels.cuda())
            
        # Zero the gradients
        optimizer.zero_grad()
        
        # Forward pass
        outputs = model(images)
        
        # Compute the loss
        loss = loss_function(outputs, labels)
        
        # Backward pass
        loss.backward()
        
        # Update the weights
        optimizer.step()
        
        # Update training loss
        train_loss += loss.cpu().data * images.size(0)
        
        # Calculate training accuracy
        _, prediction = torch.max(outputs.data, 1)
        train_accuracy += int(torch.sum(prediction == labels.data))
        
    # Calculate average training accuracy and loss
    train_accuracy = train_accuracy / train_count
    train_loss = train_loss / train_count
    # Initialize the confusion matrix
    conf_matrix = torch.zeros(5, 5)

    # Set the model to evaluation mode
    model.eval()
    
    # Initialize testing accuracy
    test_accuracy = 0.0
    
    # Iterate through the testing dataset
    for i, (images, labels) in enumerate(test_loader_32):
        # Move images and labels to GPU if available
        if torch.cuda.is_available():
            images = Variable(images.cuda())
            labels = Variable(labels.cuda())
            
        # Forward pass
        outputs = model(images)
        
        # Calculate testing accuracy
        _, prediction = torch.max(outputs.data, 1)
        test_accuracy += int(torch.sum(prediction == labels.data))
        # Update the confusion matrix
        conf_matrix += confusion_matrix(labels.cpu().numpy(), prediction.cpu().numpy(), labels=list(range(5)))
    
    # Calculate average testing accuracy
    test_accuracy = test_accuracy / test_count
    class_accuracy = conf_matrix.diag() / conf_matrix.sum(1)

    # Print class-wise accuracy
    for i in range(5):
        print(f'Class {i} Accuracy: {class_accuracy[i].item()}')
        # Print epoch-wise results
    print('Epoch: ' + str(epoch) + ' Train Loss: ' + str(train_loss) +
        ' Train Accuracy: ' + str(train_accuracy) + ' Test Accuracy: ' + str(test_accuracy))
    
    # Save the best model based on testing accuracy
    if test_accuracy > best_accuracy:
        torch.save(model.state_dict(), 'best_checkpoint.model')
        best_accuracy = test_accuracy



model for no weights no drop prob size 100

In [ ]:
# Model training and saving the best model

# Initialize the best accuracy variable
best_accuracy = 0.0

# Iterate through each epoch
for epoch in range(num_epochs):
    
    # Set the model to training mode
    model.train()
    
    # Initialize training accuracy and loss
    train_accuracy = 0.0
    train_loss = 0.0
    
    # Iterate through the training dataset
    for i, (images, labels) in enumerate(train_loader_100):
        # Move images and labels to GPU if available
        if torch.cuda.is_available():
            images = Variable(images.cuda())
            labels = Variable(labels.cuda())
            
        # Zero the gradients
        optimizer.zero_grad()
        
        # Forward pass
        outputs = model(images)
        
        # Compute the loss
        loss = loss_function(outputs, labels)
        
        # Backward pass
        loss.backward()
        
        # Update the weights
        optimizer.step()
        
        # Update training loss
        train_loss += loss.cpu().data * images.size(0)
        
        # Calculate training accuracy
        _, prediction = torch.max(outputs.data, 1)
        train_accuracy += int(torch.sum(prediction == labels.data))
        
    # Calculate average training accuracy and loss
    train_accuracy = train_accuracy / train_count
    train_loss = train_loss / train_count
    # Initialize the confusion matrix
    conf_matrix = torch.zeros(5, 5)

    # Set the model to evaluation mode
    model.eval()
    
    # Initialize testing accuracy
    test_accuracy = 0.0
    
    # Iterate through the testing dataset
    for i, (images, labels) in enumerate(test_loader_100):
        # Move images and labels to GPU if available
        if torch.cuda.is_available():
            images = Variable(images.cuda())
            labels = Variable(labels.cuda())
            
        # Forward pass
        outputs = model(images)
        
        # Calculate testing accuracy
        _, prediction = torch.max(outputs.data, 1)
        test_accuracy += int(torch.sum(prediction == labels.data))
        # Update the confusion matrix
        conf_matrix += confusion_matrix(labels.cpu().numpy(), prediction.cpu().numpy(), labels=list(range(5)))
    
    # Calculate average testing accuracy
    test_accuracy = test_accuracy / test_count
    class_accuracy = conf_matrix.diag() / conf_matrix.sum(1)

    # Print class-wise accuracy
    for i in range(5):
        print(f'Class {i} Accuracy: {class_accuracy[i].item()}')
        # Print epoch-wise results
    print('Epoch: ' + str(epoch) + ' Train Loss: ' + str(train_loss) +
        ' Train Accuracy: ' + str(train_accuracy) + ' Test Accuracy: ' + str(test_accuracy))
    
    # Save the best model based on testing accuracy
    if test_accuracy > best_accuracy:
        torch.save(model.state_dict(), 'best_checkpoint.model')
        best_accuracy = test_accuracy



In [ ]:
class ConvNet_dropout(nn.Module):
    def __init__(self, num_classes=5, dropout_prob=0.5):
        super(ConvNet_dropout, self).__init__()
        
        # Output size after convolution filter ((w-f+2P)/s) + 1
        
        # Input shape = (256, 3, 150, 150)
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=12, kernel_size=3, stride=1, padding=1)
        # Shape = (256, 12, 150, 150)
        self.bn1 = nn.BatchNorm2d(num_features=12)
        # Shape = (256, 12, 150, 150)
        self.relu1 = nn.ReLU()
        # Shape = (256, 12, 150, 150)
        self.pool = nn.MaxPool2d(kernel_size=2)
        # Reduce the image size by factor 2
        # Shape = (256, 12, 75, 75)
        
        # Dropout layer with specified dropout probability
        self.dropout1 = nn.Dropout(dropout_prob)
        
        self.conv2 = nn.Conv2d(in_channels=12, out_channels=20, kernel_size=3, stride=1, padding=1)
        # Shape = (256, 20, 75, 75)
        self.relu2 = nn.ReLU()
        # Shape = (256, 20, 75, 75)
        
        # Dropout layer with specified dropout probability
        self.dropout2 = nn.Dropout(dropout_prob)
        
        self.conv3 = nn.Conv2d(in_channels=20, out_channels=32, kernel_size=3, stride=1, padding=1)
        # Shape = (256, 32, 75, 75)
        self.bn3 = nn.BatchNorm2d(num_features=32)
        # Shape = (256, 32, 75, 75)
        self.relu3 = nn.ReLU()
        # Shape = (256, 32, 75, 75)
        
        # Dropout layer with specified dropout probability
        self.dropout3 = nn.Dropout(dropout_prob)
        
        self.fc = nn.Linear(in_features=75 * 75 * 32, out_features=num_classes)
        
    def forward(self, input):
        output = self.conv1(input)
        output = self.bn1(output)
        output = self.relu1(output)
            
        output = self.pool(output)
        output = self.dropout1(output)
            
        output = self.conv2(output)
        output = self.relu2(output)
        output = self.dropout2(output)
            
        output = self.conv3(output)
        output = self.bn3(output)
        output = self.relu3(output)
        output = self.dropout3(output)
            
        # Above output will be in matrix form, with shape (256, 32, 75, 75)
        output = output.view(-1, 32 * 75 * 75)
            
        output = self.fc(output)
            
        return output


In [ ]:
model_dropout = ConvNet_dropout(num_classes=5, dropout_prob=0.1).to(device)
optimizer_dropout=Adam(model_dropout.parameters(),lr=0.001,weight_decay=0.0001)

code for dropout prob = 0.1 no weight batch size 32

In [ ]:
# Model training and saving the best model

# Initialize the best accuracy variable
best_accuracy = 0.0

# Iterate through each epoch
for epoch in range(num_epochs):
    
    # Set the model to training mode
    model_dropout.train()
    
    # Initialize training accuracy and loss
    train_accuracy = 0.0
    train_loss = 0.0
    
    # Iterate through the training dataset
    for i, (images, labels) in enumerate(train_loader_32):
        # Move images and labels to GPU if available
        if torch.cuda.is_available():
            images = Variable(images.cuda())
            labels = Variable(labels.cuda())
            
        # Zero the gradients
        optimizer_dropout.zero_grad()
        
        # Forward pass
        outputs = model_dropout(images)
        
        # Compute the loss
        loss = loss_function(outputs, labels)
        
        # Backward pass
        loss.backward()
        
        # Update the weights
        optimizer_dropout.step()
        
        # Update training loss
        train_loss += loss.cpu().data * images.size(0)
        
        # Calculate training accuracy
        _, prediction = torch.max(outputs.data, 1)
        train_accuracy += int(torch.sum(prediction == labels.data))
        
    # Calculate average training accuracy and loss
    train_accuracy = train_accuracy / train_count
    train_loss = train_loss / train_count
    # Initialize the confusion matrix
    conf_matrix = torch.zeros(5, 5)

    # Set the model to evaluation mode
    model_dropout.eval()
    
    # Initialize testing accuracy
    test_accuracy = 0.0
    
    # Iterate through the testing dataset
    for i, (images, labels) in enumerate(test_loader_32):
        # Move images and labels to GPU if available
        if torch.cuda.is_available():
            images = Variable(images.cuda())
            labels = Variable(labels.cuda())
            
        # Forward pass
        outputs = model_dropout(images)
        
        # Calculate testing accuracy
        _, prediction = torch.max(outputs.data, 1)
        test_accuracy += int(torch.sum(prediction == labels.data))
        # Update the confusion matrix
        conf_matrix += confusion_matrix(labels.cpu().numpy(), prediction.cpu().numpy(), labels=list(range(5)))
    
    # Calculate average testing accuracy
    test_accuracy = test_accuracy / test_count
    class_accuracy = conf_matrix.diag() / conf_matrix.sum(1)

    # Print class-wise accuracy
    for i in range(5):
        print(f'Class {i} Accuracy: {class_accuracy[i].item()}')
        # scheduler.step()
        # Print epoch-wise results
    print('Epoch: ' + str(epoch) + ' Train Loss: ' + str(train_loss) +
        ' Train Accuracy: ' + str(train_accuracy) + ' Test Accuracy: ' + str(test_accuracy))
    
    # Save the best model based on testing accuracy
    if test_accuracy > best_accuracy:
        torch.save(model_dropout.state_dict(), 'best_checkpoint_dropout.model')
        best_accuracy = test_accuracy



code for dropout prob = 0.1 no weight batch size 100

In [ ]:
# Model training and saving the best model

# Initialize the best accuracy variable
best_accuracy = 0.0

# Iterate through each epoch
for epoch in range(num_epochs):
    
    # Set the model to training mode
    model_dropout.train()
    
    # Initialize training accuracy and loss
    train_accuracy = 0.0
    train_loss = 0.0
    
    # Iterate through the training dataset
    for i, (images, labels) in enumerate(train_loader_100):
        # Move images and labels to GPU if available
        if torch.cuda.is_available():
            images = Variable(images.cuda())
            labels = Variable(labels.cuda())
            
        # Zero the gradients
        optimizer_dropout.zero_grad()
        
        # Forward pass
        outputs = model_dropout(images)
        
        # Compute the loss
        loss = loss_function(outputs, labels)
        
        # Backward pass
        loss.backward()
        
        # Update the weights
        optimizer_dropout.step()
        
        # Update training loss
        train_loss += loss.cpu().data * images.size(0)
        
        # Calculate training accuracy
        _, prediction = torch.max(outputs.data, 1)
        train_accuracy += int(torch.sum(prediction == labels.data))
        
    # Calculate average training accuracy and loss
    train_accuracy = train_accuracy / train_count
    train_loss = train_loss / train_count
    # Initialize the confusion matrix
    conf_matrix = torch.zeros(5, 5)

    # Set the model to evaluation mode
    model_dropout.eval()
    
    # Initialize testing accuracy
    test_accuracy = 0.0
    
    # Iterate through the testing dataset
    for i, (images, labels) in enumerate(test_loader_100):
        # Move images and labels to GPU if available
        if torch.cuda.is_available():
            images = Variable(images.cuda())
            labels = Variable(labels.cuda())
            
        # Forward pass
        outputs = model_dropout(images)
        
        # Calculate testing accuracy
        _, prediction = torch.max(outputs.data, 1)
        test_accuracy += int(torch.sum(prediction == labels.data))
        # Update the confusion matrix
        conf_matrix += confusion_matrix(labels.cpu().numpy(), prediction.cpu().numpy(), labels=list(range(5)))
    
    # Calculate average testing accuracy
    test_accuracy = test_accuracy / test_count
    class_accuracy = conf_matrix.diag() / conf_matrix.sum(1)

    # Print class-wise accuracy
    for i in range(5):
        print(f'Class {i} Accuracy: {class_accuracy[i].item()}')
        # scheduler.step()
        # Print epoch-wise results
    print('Epoch: ' + str(epoch) + ' Train Loss: ' + str(train_loss) +
        ' Train Accuracy: ' + str(train_accuracy) + ' Test Accuracy: ' + str(test_accuracy))
    
    # Save the best model based on testing accuracy
    if test_accuracy > best_accuracy:
        torch.save(model_dropout.state_dict(), 'best_checkpoint_dropout.model')
        best_accuracy = test_accuracy



In [ ]:
model_dropout = ConvNet_dropout(num_classes=5, dropout_prob=0.2).to(device)
optimizer_dropout=Adam(model_dropout.parameters(),lr=0.001,weight_decay=0.0001)

code for dropout prob = 0.2 no weight batch size 32

In [ ]:
# Model training and saving the best model

# Initialize the best accuracy variable
best_accuracy = 0.0

# Iterate through each epoch
for epoch in range(num_epochs):
    
    # Set the model to training mode
    model_dropout.train()
    
    # Initialize training accuracy and loss
    train_accuracy = 0.0
    train_loss = 0.0
    
    # Iterate through the training dataset
    for i, (images, labels) in enumerate(train_loader_32):
        # Move images and labels to GPU if available
        if torch.cuda.is_available():
            images = Variable(images.cuda())
            labels = Variable(labels.cuda())
            
        # Zero the gradients
        optimizer_dropout.zero_grad()
        
        # Forward pass
        outputs = model_dropout(images)
        
        # Compute the loss
        loss = loss_function(outputs, labels)
        
        # Backward pass
        loss.backward()
        
        # Update the weights
        optimizer_dropout.step()
        
        # Update training loss
        train_loss += loss.cpu().data * images.size(0)
        
        # Calculate training accuracy
        _, prediction = torch.max(outputs.data, 1)
        train_accuracy += int(torch.sum(prediction == labels.data))
        
    # Calculate average training accuracy and loss
    train_accuracy = train_accuracy / train_count
    train_loss = train_loss / train_count
    # Initialize the confusion matrix
    conf_matrix = torch.zeros(5, 5)

    # Set the model to evaluation mode
    model_dropout.eval()
    
    # Initialize testing accuracy
    test_accuracy = 0.0
    
    # Iterate through the testing dataset
    for i, (images, labels) in enumerate(test_loader_32):
        # Move images and labels to GPU if available
        if torch.cuda.is_available():
            images = Variable(images.cuda())
            labels = Variable(labels.cuda())
            
        # Forward pass
        outputs = model_dropout(images)
        
        # Calculate testing accuracy
        _, prediction = torch.max(outputs.data, 1)
        test_accuracy += int(torch.sum(prediction == labels.data))
        # Update the confusion matrix
        conf_matrix += confusion_matrix(labels.cpu().numpy(), prediction.cpu().numpy(), labels=list(range(5)))
    
    # Calculate average testing accuracy
    test_accuracy = test_accuracy / test_count
    class_accuracy = conf_matrix.diag() / conf_matrix.sum(1)

    # Print class-wise accuracy
    for i in range(5):
        print(f'Class {i} Accuracy: {class_accuracy[i].item()}')
        # scheduler.step()
        # Print epoch-wise results
    print('Epoch: ' + str(epoch) + ' Train Loss: ' + str(train_loss) +
        ' Train Accuracy: ' + str(train_accuracy) + ' Test Accuracy: ' + str(test_accuracy))
    
    # Save the best model based on testing accuracy
    if test_accuracy > best_accuracy:
        torch.save(model_dropout.state_dict(), 'best_checkpoint_dropout.model')
        best_accuracy = test_accuracy



code for dropout prob = 0.2 no weight batch size 100

In [ ]:
# Model training and saving the best model

# Initialize the best accuracy variable
best_accuracy = 0.0

# Iterate through each epoch
for epoch in range(num_epochs):
    
    # Set the model to training mode
    model_dropout.train()
    
    # Initialize training accuracy and loss
    train_accuracy = 0.0
    train_loss = 0.0
    
    # Iterate through the training dataset
    for i, (images, labels) in enumerate(train_loader_100):
        # Move images and labels to GPU if available
        if torch.cuda.is_available():
            images = Variable(images.cuda())
            labels = Variable(labels.cuda())
            
        # Zero the gradients
        optimizer_dropout.zero_grad()
        
        # Forward pass
        outputs = model_dropout(images)
        
        # Compute the loss
        loss = loss_function(outputs, labels)
        
        # Backward pass
        loss.backward()
        
        # Update the weights
        optimizer_dropout.step()
        
        # Update training loss
        train_loss += loss.cpu().data * images.size(0)
        
        # Calculate training accuracy
        _, prediction = torch.max(outputs.data, 1)
        train_accuracy += int(torch.sum(prediction == labels.data))
        
    # Calculate average training accuracy and loss
    train_accuracy = train_accuracy / train_count
    train_loss = train_loss / train_count
    # Initialize the confusion matrix
    conf_matrix = torch.zeros(5, 5)

    # Set the model to evaluation mode
    model_dropout.eval()
    
    # Initialize testing accuracy
    test_accuracy = 0.0
    
    # Iterate through the testing dataset
    for i, (images, labels) in enumerate(test_loader_100):
        # Move images and labels to GPU if available
        if torch.cuda.is_available():
            images = Variable(images.cuda())
            labels = Variable(labels.cuda())
            
        # Forward pass
        outputs = model_dropout(images)
        
        # Calculate testing accuracy
        _, prediction = torch.max(outputs.data, 1)
        test_accuracy += int(torch.sum(prediction == labels.data))
        # Update the confusion matrix
        conf_matrix += confusion_matrix(labels.cpu().numpy(), prediction.cpu().numpy(), labels=list(range(5)))
    
    # Calculate average testing accuracy
    test_accuracy = test_accuracy / test_count
    class_accuracy = conf_matrix.diag() / conf_matrix.sum(1)

    # Print class-wise accuracy
    for i in range(5):
        print(f'Class {i} Accuracy: {class_accuracy[i].item()}')
        # scheduler.step()
        # Print epoch-wise results
    print('Epoch: ' + str(epoch) + ' Train Loss: ' + str(train_loss) +
        ' Train Accuracy: ' + str(train_accuracy) + ' Test Accuracy: ' + str(test_accuracy))
    
    # Save the best model based on testing accuracy
    if test_accuracy > best_accuracy:
        torch.save(model_dropout.state_dict(), 'best_checkpoint_dropout.model')
        best_accuracy = test_accuracy



In [ ]:
model_dropout = ConvNet_dropout(num_classes=5, dropout_prob=0.1).to(device)
optimizer_dropout=Adam(model_dropout.parameters(),lr=0.001,weight_decay=0.0001)

In [ ]:
# Initialize an empty list to store all labels
all_labels = []

# Iterate through the DataLoader
for _, labels in train_loader_32:
    # Append labels to the list
    all_labels.extend(labels.tolist())

# Convert the list to a tensor
all_labels_tensor = torch.tensor(all_labels)

# Calculate class weights
class_weights = 1.0 / torch.sqrt(torch.bincount(all_labels_tensor).float() / len(all_labels_tensor))

# Convert class weights to a PyTorch tensor
class_weights = torch.FloatTensor(class_weights)

# Use the calculated class weights in the CrossEntropyLoss
loss_function_weighted = nn.CrossEntropyLoss(weight=class_weights)


code for dropout prob = 0.1 with class weight batch size 32

In [ ]:
# Model training and saving the best model

# Initialize the best accuracy variable
best_accuracy = 0.0

# Iterate through each epoch
for epoch in range(num_epochs):
    
    # Set the model to training mode
    model_dropout.train()
    
    # Initialize training accuracy and loss
    train_accuracy = 0.0
    train_loss = 0.0
    
    # Iterate through the training dataset
    for i, (images, labels) in enumerate(train_loader_32):
        # Move images and labels to GPU if available
        if torch.cuda.is_available():
            images = Variable(images.cuda())
            labels = Variable(labels.cuda())
            
        # Zero the gradients
        optimizer_dropout.zero_grad()
        
        # Forward pass
        outputs = model_dropout(images)
        
        # Compute the loss
        loss = loss_function_weighted(outputs, labels)
        
        # Backward pass
        loss.backward()
        
        # Update the weights
        optimizer_dropout.step()
        
        # Update training loss
        train_loss += loss.cpu().data * images.size(0)
        
        # Calculate training accuracy
        _, prediction = torch.max(outputs.data, 1)
        train_accuracy += int(torch.sum(prediction == labels.data))
        
    # Calculate average training accuracy and loss
    train_accuracy = train_accuracy / train_count
    train_loss = train_loss / train_count
    # Initialize the confusion matrix
    conf_matrix = torch.zeros(5, 5)

    # Set the model to evaluation mode
    model_dropout.eval()
    
    # Initialize testing accuracy
    test_accuracy = 0.0
    
    # Iterate through the testing dataset
    for i, (images, labels) in enumerate(test_loader_32):
        # Move images and labels to GPU if available
        if torch.cuda.is_available():
            images = Variable(images.cuda())
            labels = Variable(labels.cuda())
            
        # Forward pass
        outputs = model_dropout(images)
        
        # Calculate testing accuracy
        _, prediction = torch.max(outputs.data, 1)
        test_accuracy += int(torch.sum(prediction == labels.data))
        # Update the confusion matrix
        conf_matrix += confusion_matrix(labels.cpu().numpy(), prediction.cpu().numpy(), labels=list(range(5)))
    
    # Calculate average testing accuracy
    test_accuracy = test_accuracy / test_count
    class_accuracy = conf_matrix.diag() / conf_matrix.sum(1)

    # Print class-wise accuracy
    for i in range(5):
        print(f'Class {i} Accuracy: {class_accuracy[i].item()}')
        # scheduler.step()
        # Print epoch-wise results
    print('Epoch: ' + str(epoch) + ' Train Loss: ' + str(train_loss) +
        ' Train Accuracy: ' + str(train_accuracy) + ' Test Accuracy: ' + str(test_accuracy))
    
    # Save the best model based on testing accuracy
    if test_accuracy > best_accuracy:
        torch.save(model_dropout.state_dict(), 'best_checkpoint_dropout.model')
        best_accuracy = test_accuracy



code for dropout prob = 0.1 with class weight batch size 100

In [ ]:
# Initialize an empty list to store all labels
all_labels = []

# Iterate through the DataLoader
for _, labels in train_loader_100:
    # Append labels to the list
    all_labels.extend(labels.tolist())

# Convert the list to a tensor
all_labels_tensor = torch.tensor(all_labels)

# Calculate class weights
class_weights = 1.0 / torch.sqrt(torch.bincount(all_labels_tensor).float() / len(all_labels_tensor))

# Convert class weights to a PyTorch tensor
class_weights = torch.FloatTensor(class_weights)

# Use the calculated class weights in the CrossEntropyLoss
loss_function_weighted = nn.CrossEntropyLoss(weight=class_weights)


In [ ]:
# Model training and saving the best model

# Initialize the best accuracy variable
best_accuracy = 0.0

# Iterate through each epoch
for epoch in range(num_epochs):
    
    # Set the model to training mode
    model_dropout.train()
    
    # Initialize training accuracy and loss
    train_accuracy = 0.0
    train_loss = 0.0
    
    # Iterate through the training dataset
    for i, (images, labels) in enumerate(train_loader_100):
        # Move images and labels to GPU if available
        if torch.cuda.is_available():
            images = Variable(images.cuda())
            labels = Variable(labels.cuda())
            
        # Zero the gradients
        optimizer_dropout.zero_grad()
        
        # Forward pass
        outputs = model_dropout(images)
        
        # Compute the loss
        loss = loss_function_weighted(outputs, labels)
        
        # Backward pass
        loss.backward()
        
        # Update the weights
        optimizer_dropout.step()
        
        # Update training loss
        train_loss += loss.cpu().data * images.size(0)
        
        # Calculate training accuracy
        _, prediction = torch.max(outputs.data, 1)
        train_accuracy += int(torch.sum(prediction == labels.data))
        
    # Calculate average training accuracy and loss
    train_accuracy = train_accuracy / train_count
    train_loss = train_loss / train_count
    # Initialize the confusion matrix
    conf_matrix = torch.zeros(5, 5)

    # Set the model to evaluation mode
    model_dropout.eval()
    
    # Initialize testing accuracy
    test_accuracy = 0.0
    
    # Iterate through the testing dataset
    for i, (images, labels) in enumerate(test_loader_100):
        # Move images and labels to GPU if available
        if torch.cuda.is_available():
            images = Variable(images.cuda())
            labels = Variable(labels.cuda())
            
        # Forward pass
        outputs = model_dropout(images)
        
        # Calculate testing accuracy
        _, prediction = torch.max(outputs.data, 1)
        test_accuracy += int(torch.sum(prediction == labels.data))
        # Update the confusion matrix
        conf_matrix += confusion_matrix(labels.cpu().numpy(), prediction.cpu().numpy(), labels=list(range(5)))
    
    # Calculate average testing accuracy
    test_accuracy = test_accuracy / test_count
    class_accuracy = conf_matrix.diag() / conf_matrix.sum(1)

    # Print class-wise accuracy
    for i in range(5):
        print(f'Class {i} Accuracy: {class_accuracy[i].item()}')
        # scheduler.step()
        # Print epoch-wise results
    print('Epoch: ' + str(epoch) + ' Train Loss: ' + str(train_loss) +
        ' Train Accuracy: ' + str(train_accuracy) + ' Test Accuracy: ' + str(test_accuracy))
    
    # Save the best model based on testing accuracy
    if test_accuracy > best_accuracy:
        torch.save(model_dropout.state_dict(), 'best_checkpoint_dropout.model')
        best_accuracy = test_accuracy



code for dropout prob = 0.1 with class weight batch size 100 learning rate = 0.01

In [ ]:

optimizer_dropout=Adam(model_dropout.parameters(),lr=0.01,weight_decay=0.0001)

In [ ]:
# Model training and saving the best model

# Initialize the best accuracy variable
best_accuracy = 0.0

# Iterate through each epoch
for epoch in range(num_epochs):
    
    # Set the model to training mode
    model_dropout.train()
    
    # Initialize training accuracy and loss
    train_accuracy = 0.0
    train_loss = 0.0
    
    # Iterate through the training dataset
    for i, (images, labels) in enumerate(train_loader_100):
        # Move images and labels to GPU if available
        if torch.cuda.is_available():
            images = Variable(images.cuda())
            labels = Variable(labels.cuda())
            
        # Zero the gradients
        optimizer_dropout.zero_grad()
        
        # Forward pass
        outputs = model_dropout(images)
        
        # Compute the loss
        loss = loss_function(outputs, labels)
        
        # Backward pass
        loss.backward()
        
        # Update the weights
        optimizer_dropout.step()
        
        # Update training loss
        train_loss += loss.cpu().data * images.size(0)
        
        # Calculate training accuracy
        _, prediction = torch.max(outputs.data, 1)
        train_accuracy += int(torch.sum(prediction == labels.data))
        
    # Calculate average training accuracy and loss
    train_accuracy = train_accuracy / train_count
    train_loss = train_loss / train_count
    # Initialize the confusion matrix
    conf_matrix = torch.zeros(5, 5)

    # Set the model to evaluation mode
    model_dropout.eval()
    
    # Initialize testing accuracy
    test_accuracy = 0.0
    
    # Iterate through the testing dataset
    for i, (images, labels) in enumerate(test_loader_100):
        # Move images and labels to GPU if available
        if torch.cuda.is_available():
            images = Variable(images.cuda())
            labels = Variable(labels.cuda())
            
        # Forward pass
        outputs = model_dropout(images)
        
        # Calculate testing accuracy
        _, prediction = torch.max(outputs.data, 1)
        test_accuracy += int(torch.sum(prediction == labels.data))
        # Update the confusion matrix
        conf_matrix += confusion_matrix(labels.cpu().numpy(), prediction.cpu().numpy(), labels=list(range(5)))
    
    # Calculate average testing accuracy
    test_accuracy = test_accuracy / test_count
    class_accuracy = conf_matrix.diag() / conf_matrix.sum(1)

    # Print class-wise accuracy
    for i in range(5):
        print(f'Class {i} Accuracy: {class_accuracy[i].item()}')
        # scheduler.step()
        # Print epoch-wise results
    print('Epoch: ' + str(epoch) + ' Train Loss: ' + str(train_loss) +
        ' Train Accuracy: ' + str(train_accuracy) + ' Test Accuracy: ' + str(test_accuracy))
    
    # Save the best model based on testing accuracy
    if test_accuracy > best_accuracy:
        torch.save(model_dropout.state_dict(), 'best_checkpoint_dropout.model')
        best_accuracy = test_accuracy



In [ ]:
class ConvNet_dropout_tanh(nn.Module):
    def __init__(self, num_classes=5, dropout_prob=0.5):
        super(ConvNet_dropout_tanh, self).__init__()
        
        # Output size after convolution filter ((w-f+2P)/s) + 1
        
        # Input shape = (256, 3, 150, 150)
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=12, kernel_size=3, stride=1, padding=1)
        # Shape = (256, 12, 150, 150)
        self.bn1 = nn.BatchNorm2d(num_features=12)
        # Shape = (256, 12, 150, 150)
        self.tanh1 = nn.Tanh()
        # Shape = (256, 12, 150, 150)
        self.pool = nn.MaxPool2d(kernel_size=2)
        # Reduce the image size by factor 2
        # Shape = (256, 12, 75, 75)
        
        # Dropout layer with specified dropout probability
        self.dropout1 = nn.Dropout(dropout_prob)
        
        self.conv2 = nn.Conv2d(in_channels=12, out_channels=20, kernel_size=3, stride=1, padding=1)
        # Shape = (256, 20, 75, 75)
        self.tanh2 = nn.Tanh()
        # Shape = (256, 20, 75, 75)
        
        # Dropout layer with specified dropout probability
        self.dropout2 = nn.Dropout(dropout_prob)
        
        self.conv3 = nn.Conv2d(in_channels=20, out_channels=32, kernel_size=3, stride=1, padding=1)
        # Shape = (256, 32, 75, 75)
        self.bn3 = nn.BatchNorm2d(num_features=32)
        # Shape = (256, 32, 75, 75)
        self.tanh3 = nn.Tanh()
        # Shape = (256, 32, 75, 75)
        
        # Dropout layer with specified dropout probability
        self.dropout3 = nn.Dropout(dropout_prob)
        
        self.fc = nn.Linear(in_features=75 * 75 * 32, out_features=num_classes)
        
    def forward(self, input):
        output = self.conv1(input)
        output = self.bn1(output)
        output = self.tanh1(output)
            
        output = self.pool(output)
        output = self.dropout1(output)
            
        output = self.conv2(output)
        output = self.tanh2(output)
        output = self.dropout2(output)
            
        output = self.conv3(output)
        output = self.bn3(output)
        output = self.tanh3(output)
        output = self.dropout3(output)
            
        # Above output will be in matrix form, with shape (256, 32, 75, 75)
        output = output.view(-1, 32 * 75 * 75)
            
        output = self.fc(output)
            
        return output


In [ ]:
model_dropout_tanh = ConvNet_dropout_tanh(num_classes=5, dropout_prob=0.1).to(device)
optimizer_dropout_tanh=Adam(model_dropout_tanh.parameters(),lr=0.001,weight_decay=0.0001)

code for dropout prob = 0.1 no class weight batch size 100 learning rate = 0.001 tanh

In [ ]:
# Model training and saving the best model

# Initialize the best accuracy variable
best_accuracy = 0.0

# Iterate through each epoch
for epoch in range(num_epochs):
    
    # Set the model to training mode
    model_dropout_tanh.train()
    
    # Initialize training accuracy and loss
    train_accuracy = 0.0
    train_loss = 0.0
    
    # Iterate through the training dataset
    for i, (images, labels) in enumerate(train_loader_100):
        # Move images and labels to GPU if available
        if torch.cuda.is_available():
            images = Variable(images.cuda())
            labels = Variable(labels.cuda())
            
        # Zero the gradients
        optimizer_dropout_tanh.zero_grad()
        
        # Forward pass
        outputs = model_dropout_tanh(images)
        
        # Compute the loss
        loss = loss_function(outputs, labels)
        
        # Backward pass
        loss.backward()
        
        # Update the weights
        optimizer_dropout_tanh.step()
        
        # Update training loss
        train_loss += loss.cpu().data * images.size(0)
        
        # Calculate training accuracy
        _, prediction = torch.max(outputs.data, 1)
        train_accuracy += int(torch.sum(prediction == labels.data))
        
    # Calculate average training accuracy and loss
    train_accuracy = train_accuracy / train_count
    train_loss = train_loss / train_count
    # Initialize the confusion matrix
    conf_matrix = torch.zeros(5, 5)

    # Set the model to evaluation mode
    model_dropout_tanh.eval()
    
    # Initialize testing accuracy
    test_accuracy = 0.0
    
    # Iterate through the testing dataset
    for i, (images, labels) in enumerate(test_loader_100):
        # Move images and labels to GPU if available
        if torch.cuda.is_available():
            images = Variable(images.cuda())
            labels = Variable(labels.cuda())
            
        # Forward pass
        outputs = model_dropout_tanh(images)
        
        # Calculate testing accuracy
        _, prediction = torch.max(outputs.data, 1)
        test_accuracy += int(torch.sum(prediction == labels.data))
        # Update the confusion matrix
        conf_matrix += confusion_matrix(labels.cpu().numpy(), prediction.cpu().numpy(), labels=list(range(5)))
    
    # Calculate average testing accuracy
    test_accuracy = test_accuracy / test_count
    class_accuracy = conf_matrix.diag() / conf_matrix.sum(1)

    # Print class-wise accuracy
    for i in range(5):
        print(f'Class {i} Accuracy: {class_accuracy[i].item()}')
        # scheduler.step()
        # Print epoch-wise results
    print('Epoch: ' + str(epoch) + ' Train Loss: ' + str(train_loss) +
        ' Train Accuracy: ' + str(train_accuracy) + ' Test Accuracy: ' + str(test_accuracy))
    
    # Save the best model based on testing accuracy
    if test_accuracy > best_accuracy:
        torch.save(model_dropout_tanh.state_dict(), 'best_checkpoint_dropout.model')
        best_accuracy = test_accuracy



code for dropout prob = 0.1 no class weight batch size 32 learning rate = 0.001 tanh

In [ ]:
# Model training and saving the best model

# Initialize the best accuracy variable
best_accuracy = 0.0

# Iterate through each epoch
for epoch in range(num_epochs):
    
    # Set the model to training mode
    model_dropout_tanh.train()
    
    # Initialize training accuracy and loss
    train_accuracy = 0.0
    train_loss = 0.0
    
    # Iterate through the training dataset
    for i, (images, labels) in enumerate(train_loader_32):
        # Move images and labels to GPU if available
        if torch.cuda.is_available():
            images = Variable(images.cuda())
            labels = Variable(labels.cuda())
            
        # Zero the gradients
        optimizer_dropout_tanh.zero_grad()
        
        # Forward pass
        outputs = model_dropout_tanh(images)
        
        # Compute the loss
        loss = loss_function(outputs, labels)
        
        # Backward pass
        loss.backward()
        
        # Update the weights
        optimizer_dropout_tanh.step()
        
        # Update training loss
        train_loss += loss.cpu().data * images.size(0)
        
        # Calculate training accuracy
        _, prediction = torch.max(outputs.data, 1)
        train_accuracy += int(torch.sum(prediction == labels.data))
        
    # Calculate average training accuracy and loss
    train_accuracy = train_accuracy / train_count
    train_loss = train_loss / train_count
    # Initialize the confusion matrix
    conf_matrix = torch.zeros(5, 5)

    # Set the model to evaluation mode
    model_dropout_tanh.eval()
    
    # Initialize testing accuracy
    test_accuracy = 0.0
    
    # Iterate through the testing dataset
    for i, (images, labels) in enumerate(test_loader_32):
        # Move images and labels to GPU if available
        if torch.cuda.is_available():
            images = Variable(images.cuda())
            labels = Variable(labels.cuda())
            
        # Forward pass
        outputs = model_dropout_tanh(images)
        
        # Calculate testing accuracy
        _, prediction = torch.max(outputs.data, 1)
        test_accuracy += int(torch.sum(prediction == labels.data))
        # Update the confusion matrix
        conf_matrix += confusion_matrix(labels.cpu().numpy(), prediction.cpu().numpy(), labels=list(range(5)))
    
    # Calculate average testing accuracy
    test_accuracy = test_accuracy / test_count
    class_accuracy = conf_matrix.diag() / conf_matrix.sum(1)

    # Print class-wise accuracy
    for i in range(5):
        print(f'Class {i} Accuracy: {class_accuracy[i].item()}')
        # scheduler.step()
        # Print epoch-wise results
    print('Epoch: ' + str(epoch) + ' Train Loss: ' + str(train_loss) +
        ' Train Accuracy: ' + str(train_accuracy) + ' Test Accuracy: ' + str(test_accuracy))
    
    # Save the best model based on testing accuracy
    if test_accuracy > best_accuracy:
        torch.save(model_dropout_tanh.state_dict(), 'best_checkpoint_dropout.model')
        best_accuracy = test_accuracy



In [ ]:
class ConvNet_dropout_avgpool(nn.Module):
    def __init__(self, num_classes=5, dropout_prob=0.5):
        super(ConvNet_dropout_avgpool, self).__init__()
        
        # Output size after convolution filter ((w-f+2P)/s) + 1
        
        # Input shape = (256, 3, 150, 150)
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=12, kernel_size=3, stride=1, padding=1)
        # Shape = (256, 12, 150, 150)
        self.bn1 = nn.BatchNorm2d(num_features=12)
        # Shape = (256, 12, 150, 150)
        self.relu1 = nn.ReLU()
        # Shape = (256, 12, 150, 150)
        self.pool = nn.AvgPool2d(kernel_size=2)
        # Reduce the image size by factor 2
        # Shape = (256, 12, 75, 75)
        
        # Dropout layer with specified dropout probability
        self.dropout1 = nn.Dropout(dropout_prob)
        
        self.conv2 = nn.Conv2d(in_channels=12, out_channels=20, kernel_size=3, stride=1, padding=1)
        # Shape = (256, 20, 75, 75)
        self.relu2 = nn.ReLU()
        # Shape = (256, 20, 75, 75)
        
        # Dropout layer with specified dropout probability
        self.dropout2 = nn.Dropout(dropout_prob)
        
        self.conv3 = nn.Conv2d(in_channels=20, out_channels=32, kernel_size=3, stride=1, padding=1)
        # Shape = (256, 32, 75, 75)
        self.bn3 = nn.BatchNorm2d(num_features=32)
        # Shape = (256, 32, 75, 75)
        self.relu3 = nn.ReLU()
        # Shape = (256, 32, 75, 75)
        
        # Dropout layer with specified dropout probability
        self.dropout3 = nn.Dropout(dropout_prob)
        
        self.fc = nn.Linear(in_features=75 * 75 * 32, out_features=num_classes)
        
    def forward(self, input):
        output = self.conv1(input)
        output = self.bn1(output)
        output = self.relu1(output)
            
        output = self.pool(output)
        output = self.dropout1(output)
            
        output = self.conv2(output)
        output = self.relu2(output)
        output = self.dropout2(output)
            
        output = self.conv3(output)
        output = self.bn3(output)
        output = self.relu3(output)
        output = self.dropout3(output)
            
        # Above output will be in matrix form, with shape (256, 32, 75, 75)
        output = output.view(-1, 32 * 75 * 75)
            
        output = self.fc(output)
            
        return output


In [ ]:
model_dropout_avgpool = ConvNet_dropout_avgpool(num_classes=5, dropout_prob=0.1).to(device)
optimizer_dropout_avgpool=Adam(model_dropout_avgpool.parameters(),lr=0.001,weight_decay=0.0001)

code for dropout prob = 0.1 with class weight batch size 32 learning rate = 0.001 avg pooling

In [ ]:
# Model training and saving the best model

# Initialize the best accuracy variable
best_accuracy = 0.0

# Iterate through each epoch
for epoch in range(num_epochs):
    
    # Set the model to training mode
    model_dropout_avgpool.train()
    
    # Initialize training accuracy and loss
    train_accuracy = 0.0
    train_loss = 0.0
    
    # Iterate through the training dataset
    for i, (images, labels) in enumerate(train_loader_32):
        # Move images and labels to GPU if available
        if torch.cuda.is_available():
            images = Variable(images.cuda())
            labels = Variable(labels.cuda())
            
        # Zero the gradients
        optimizer_dropout_avgpool.zero_grad()
        
        # Forward pass
        outputs = model_dropout_avgpool(images)
        
        # Compute the loss
        loss = loss_function_weighted(outputs, labels)
        
        # Backward pass
        loss.backward()
        
        # Update the weights
        optimizer_dropout_avgpool.step()
        
        # Update training loss
        train_loss += loss.cpu().data * images.size(0)
        
        # Calculate training accuracy
        _, prediction = torch.max(outputs.data, 1)
        train_accuracy += int(torch.sum(prediction == labels.data))
        
    # Calculate average training accuracy and loss
    train_accuracy = train_accuracy / train_count
    train_loss = train_loss / train_count
    # Initialize the confusion matrix
    conf_matrix = torch.zeros(5, 5)

    # Set the model to evaluation mode
    model_dropout_avgpool.eval()
    
    # Initialize testing accuracy
    test_accuracy = 0.0
    
    # Iterate through the testing dataset
    for i, (images, labels) in enumerate(test_loader_32):
        # Move images and labels to GPU if available
        if torch.cuda.is_available():
            images = Variable(images.cuda())
            labels = Variable(labels.cuda())
            
        # Forward pass
        outputs = model_dropout_avgpool(images)
        
        # Calculate testing accuracy
        _, prediction = torch.max(outputs.data, 1)
        test_accuracy += int(torch.sum(prediction == labels.data))
        # Update the confusion matrix
        conf_matrix += confusion_matrix(labels.cpu().numpy(), prediction.cpu().numpy(), labels=list(range(5)))
    
    # Calculate average testing accuracy
    test_accuracy = test_accuracy / test_count
    class_accuracy = conf_matrix.diag() / conf_matrix.sum(1)

    # Print class-wise accuracy
    for i in range(5):
        print(f'Class {i} Accuracy: {class_accuracy[i].item()}')
        # scheduler.step()
        # Print epoch-wise results
    print('Epoch: ' + str(epoch) + ' Train Loss: ' + str(train_loss) +
        ' Train Accuracy: ' + str(train_accuracy) + ' Test Accuracy: ' + str(test_accuracy))
    
    # Save the best model based on testing accuracy
    if test_accuracy > best_accuracy:
        torch.save(model_dropout_avgpool.state_dict(), 'best_checkpoint_dropout.model')
        best_accuracy = test_accuracy



code for dropout prob = 0.1 with class weight batch size 100 learning rate = 0.001 avg pooling

In [ ]:
# Model training and saving the best model

# Initialize the best accuracy variable
best_accuracy = 0.0

# Iterate through each epoch
for epoch in range(num_epochs):
    
    # Set the model to training mode
    model_dropout_avgpool.train()
    
    # Initialize training accuracy and loss
    train_accuracy = 0.0
    train_loss = 0.0
    
    # Iterate through the training dataset
    for i, (images, labels) in enumerate(train_loader_100):
        # Move images and labels to GPU if available
        if torch.cuda.is_available():
            images = Variable(images.cuda())
            labels = Variable(labels.cuda())
            
        # Zero the gradients
        optimizer_dropout_avgpool.zero_grad()
        
        # Forward pass
        outputs = model_dropout_avgpool(images)
        
        # Compute the loss
        loss = loss_function_weighted(outputs, labels)
        
        # Backward pass
        loss.backward()
        
        # Update the weights
        optimizer_dropout_avgpool.step()
        
        # Update training loss
        train_loss += loss.cpu().data * images.size(0)
        
        # Calculate training accuracy
        _, prediction = torch.max(outputs.data, 1)
        train_accuracy += int(torch.sum(prediction == labels.data))
        
    # Calculate average training accuracy and loss
    train_accuracy = train_accuracy / train_count
    train_loss = train_loss / train_count
    # Initialize the confusion matrix
    conf_matrix = torch.zeros(5, 5)

    # Set the model to evaluation mode
    model_dropout_avgpool.eval()
    
    # Initialize testing accuracy
    test_accuracy = 0.0
    
    # Iterate through the testing dataset
    for i, (images, labels) in enumerate(test_loader_100):
        # Move images and labels to GPU if available
        if torch.cuda.is_available():
            images = Variable(images.cuda())
            labels = Variable(labels.cuda())
            
        # Forward pass
        outputs = model_dropout_avgpool(images)
        
        # Calculate testing accuracy
        _, prediction = torch.max(outputs.data, 1)
        test_accuracy += int(torch.sum(prediction == labels.data))
        # Update the confusion matrix
        conf_matrix += confusion_matrix(labels.cpu().numpy(), prediction.cpu().numpy(), labels=list(range(5)))
    
    # Calculate average testing accuracy
    test_accuracy = test_accuracy / test_count
    class_accuracy = conf_matrix.diag() / conf_matrix.sum(1)

    # Print class-wise accuracy
    for i in range(5):
        print(f'Class {i} Accuracy: {class_accuracy[i].item()}')
        # scheduler.step()
        # Print epoch-wise results
    print('Epoch: ' + str(epoch) + ' Train Loss: ' + str(train_loss) +
        ' Train Accuracy: ' + str(train_accuracy) + ' Test Accuracy: ' + str(test_accuracy))
    
    # Save the best model based on testing accuracy
    if test_accuracy > best_accuracy:
        torch.save(model_dropout_avgpool.state_dict(), 'best_checkpoint_dropout.model')
        best_accuracy = test_accuracy



code for dropout prob = 0.1 no class weight batch size 32 learning rate = 0.001 avg pooling

In [ ]:
# Model training and saving the best model

# Initialize the best accuracy variable
best_accuracy = 0.0

# Iterate through each epoch
for epoch in range(num_epochs):
    
    # Set the model to training mode
    model_dropout_avgpool.train()
    
    # Initialize training accuracy and loss
    train_accuracy = 0.0
    train_loss = 0.0
    
    # Iterate through the training dataset
    for i, (images, labels) in enumerate(train_loader_32):
        # Move images and labels to GPU if available
        if torch.cuda.is_available():
            images = Variable(images.cuda())
            labels = Variable(labels.cuda())
            
        # Zero the gradients
        optimizer_dropout_avgpool.zero_grad()
        
        # Forward pass
        outputs = model_dropout_avgpool(images)
        
        # Compute the loss
        loss = loss_function(outputs, labels)
        
        # Backward pass
        loss.backward()
        
        # Update the weights
        optimizer_dropout_avgpool.step()
        
        # Update training loss
        train_loss += loss.cpu().data * images.size(0)
        
        # Calculate training accuracy
        _, prediction = torch.max(outputs.data, 1)
        train_accuracy += int(torch.sum(prediction == labels.data))
        
    # Calculate average training accuracy and loss
    train_accuracy = train_accuracy / train_count
    train_loss = train_loss / train_count
    # Initialize the confusion matrix
    conf_matrix = torch.zeros(5, 5)

    # Set the model to evaluation mode
    model_dropout_avgpool.eval()
    
    # Initialize testing accuracy
    test_accuracy = 0.0
    
    # Iterate through the testing dataset
    for i, (images, labels) in enumerate(test_loader_32):
        # Move images and labels to GPU if available
        if torch.cuda.is_available():
            images = Variable(images.cuda())
            labels = Variable(labels.cuda())
            
        # Forward pass
        outputs = model_dropout_avgpool(images)
        
        # Calculate testing accuracy
        _, prediction = torch.max(outputs.data, 1)
        test_accuracy += int(torch.sum(prediction == labels.data))
        # Update the confusion matrix
        conf_matrix += confusion_matrix(labels.cpu().numpy(), prediction.cpu().numpy(), labels=list(range(5)))
    
    # Calculate average testing accuracy
    test_accuracy = test_accuracy / test_count
    class_accuracy = conf_matrix.diag() / conf_matrix.sum(1)

    # Print class-wise accuracy
    for i in range(5):
        print(f'Class {i} Accuracy: {class_accuracy[i].item()}')
        # scheduler.step()
        # Print epoch-wise results
    print('Epoch: ' + str(epoch) + ' Train Loss: ' + str(train_loss) +
        ' Train Accuracy: ' + str(train_accuracy) + ' Test Accuracy: ' + str(test_accuracy))
    
    # Save the best model based on testing accuracy
    if test_accuracy > best_accuracy:
        torch.save(model_dropout_avgpool.state_dict(), 'best_checkpoint_dropout.model')
        best_accuracy = test_accuracy



code for dropout prob = 0.1 no class weight batch size 100 learning rate = 0.001 avg pooling

In [ ]:
# Model training and saving the best model

# Initialize the best accuracy variable
best_accuracy = 0.0

# Iterate through each epoch
for epoch in range(num_epochs):
    
    # Set the model to training mode
    model_dropout_avgpool.train()
    
    # Initialize training accuracy and loss
    train_accuracy = 0.0
    train_loss = 0.0
    
    # Iterate through the training dataset
    for i, (images, labels) in enumerate(train_loader_100):
        # Move images and labels to GPU if available
        if torch.cuda.is_available():
            images = Variable(images.cuda())
            labels = Variable(labels.cuda())
            
        # Zero the gradients
        optimizer_dropout_avgpool.zero_grad()
        
        # Forward pass
        outputs = model_dropout_avgpool(images)
        
        # Compute the loss
        loss = loss_function_weighted(outputs, labels)
        
        # Backward pass
        loss.backward()
        
        # Update the weights
        optimizer_dropout_avgpool.step()
        
        # Update training loss
        train_loss += loss.cpu().data * images.size(0)
        
        # Calculate training accuracy
        _, prediction = torch.max(outputs.data, 1)
        train_accuracy += int(torch.sum(prediction == labels.data))
        
    # Calculate average training accuracy and loss
    train_accuracy = train_accuracy / train_count
    train_loss = train_loss / train_count
    # Initialize the confusion matrix
    conf_matrix = torch.zeros(5, 5)

    # Set the model to evaluation mode
    model_dropout_avgpool.eval()
    
    # Initialize testing accuracy
    test_accuracy = 0.0
    
    # Iterate through the testing dataset
    for i, (images, labels) in enumerate(test_loader_100):
        # Move images and labels to GPU if available
        if torch.cuda.is_available():
            images = Variable(images.cuda())
            labels = Variable(labels.cuda())
            
        # Forward pass
        outputs = model_dropout_avgpool(images)
        
        # Calculate testing accuracy
        _, prediction = torch.max(outputs.data, 1)
        test_accuracy += int(torch.sum(prediction == labels.data))
        # Update the confusion matrix
        conf_matrix += confusion_matrix(labels.cpu().numpy(), prediction.cpu().numpy(), labels=list(range(5)))
    
    # Calculate average testing accuracy
    test_accuracy = test_accuracy / test_count
    class_accuracy = conf_matrix.diag() / conf_matrix.sum(1)

    # Print class-wise accuracy
    for i in range(5):
        print(f'Class {i} Accuracy: {class_accuracy[i].item()}')
        # scheduler.step()
        # Print epoch-wise results
    print('Epoch: ' + str(epoch) + ' Train Loss: ' + str(train_loss) +
        ' Train Accuracy: ' + str(train_accuracy) + ' Test Accuracy: ' + str(test_accuracy))
    
    # Save the best model based on testing accuracy
    if test_accuracy > best_accuracy:
        torch.save(model_dropout_avgpool.state_dict(), 'best_checkpoint_dropout.model')
        best_accuracy = test_accuracy



In [ ]:
model_dropout_avgpool = ConvNet_dropout_avgpool(num_classes=5, dropout_prob=0.2).to(device)
optimizer_dropout_avgpool=Adam(model_dropout_avgpool.parameters(),lr=0.001,weight_decay=0.0001)

code for dropout prob = 0.2 no class weight batch size 100 learning rate = 0.001 avg pooling

In [ ]:
# Model training and saving the best model

# Initialize the best accuracy variable
best_accuracy = 0.0

# Iterate through each epoch
for epoch in range(num_epochs):
    
    # Set the model to training mode
    model_dropout_avgpool.train()
    
    # Initialize training accuracy and loss
    train_accuracy = 0.0
    train_loss = 0.0
    
    # Iterate through the training dataset
    for i, (images, labels) in enumerate(train_loader_100):
        # Move images and labels to GPU if available
        if torch.cuda.is_available():
            images = Variable(images.cuda())
            labels = Variable(labels.cuda())
            
        # Zero the gradients
        optimizer_dropout_avgpool.zero_grad()
        
        # Forward pass
        outputs = model_dropout_avgpool(images)
        
        # Compute the loss
        loss = loss_function(outputs, labels)
        
        # Backward pass
        loss.backward()
        
        # Update the weights
        optimizer_dropout_avgpool.step()
        
        # Update training loss
        train_loss += loss.cpu().data * images.size(0)
        
        # Calculate training accuracy
        _, prediction = torch.max(outputs.data, 1)
        train_accuracy += int(torch.sum(prediction == labels.data))
        
    # Calculate average training accuracy and loss
    train_accuracy = train_accuracy / train_count
    train_loss = train_loss / train_count
    # Initialize the confusion matrix
    conf_matrix = torch.zeros(5, 5)

    # Set the model to evaluation mode
    model_dropout_avgpool.eval()
    
    # Initialize testing accuracy
    test_accuracy = 0.0
    
    # Iterate through the testing dataset
    for i, (images, labels) in enumerate(test_loader_100):
        # Move images and labels to GPU if available
        if torch.cuda.is_available():
            images = Variable(images.cuda())
            labels = Variable(labels.cuda())
            
        # Forward pass
        outputs = model_dropout_avgpool(images)
        
        # Calculate testing accuracy
        _, prediction = torch.max(outputs.data, 1)
        test_accuracy += int(torch.sum(prediction == labels.data))
        # Update the confusion matrix
        conf_matrix += confusion_matrix(labels.cpu().numpy(), prediction.cpu().numpy(), labels=list(range(5)))
    
    # Calculate average testing accuracy
    test_accuracy = test_accuracy / test_count
    class_accuracy = conf_matrix.diag() / conf_matrix.sum(1)

    # Print class-wise accuracy
    for i in range(5):
        print(f'Class {i} Accuracy: {class_accuracy[i].item()}')
        # scheduler.step()
        # Print epoch-wise results
    print('Epoch: ' + str(epoch) + ' Train Loss: ' + str(train_loss) +
        ' Train Accuracy: ' + str(train_accuracy) + ' Test Accuracy: ' + str(test_accuracy))
    
    # Save the best model based on testing accuracy
    if test_accuracy > best_accuracy:
        torch.save(model_dropout_avgpool.state_dict(), 'best_checkpoint_dropout.model')
        best_accuracy = test_accuracy



In [ ]:
class ConvNet_complex(nn.Module):
    def __init__(self, num_classes=5, dropout_prob=0.3):
        super(ConvNet_complex, self).__init__()
        
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(num_features=32)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(kernel_size=2)
        self.dropout1 = nn.Dropout(dropout_prob)
        
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(num_features=64)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(kernel_size=2)
        self.dropout2 = nn.Dropout(dropout_prob)
        
        self.conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1)
        self.bn3 = nn.BatchNorm2d(num_features=128)
        self.relu3 = nn.ReLU()
        self.pool3 = nn.MaxPool2d(kernel_size=2)
        self.dropout3 = nn.Dropout(dropout_prob)
        
        self.fc1 = nn.Linear(in_features=128 * 18 * 18, out_features=512)
        self.relu_fc1 = nn.ReLU()
        self.dropout_fc1 = nn.Dropout(dropout_prob)
        
        self.fc2 = nn.Linear(in_features=512, out_features=num_classes)
        
    def forward(self, input):
        output = self.conv1(input)
        output = self.bn1(output)
        output = self.relu1(output)
        output = self.pool1(output)
        output = self.dropout1(output)
        
        output = self.conv2(output)
        output = self.bn2(output)
        output = self.relu2(output)
        output = self.pool2(output)
        output = self.dropout2(output)
        
        output = self.conv3(output)
        output = self.bn3(output)
        output = self.relu3(output)
        output = self.pool3(output)
        output = self.dropout3(output)
        
        output = output.view(-1, 128 * 18 * 18)
        
        output = self.fc1(output)
        output = self.relu_fc1(output)
        output = self.dropout_fc1(output)
        
        output = self.fc2(output)
        
        return output


In [ ]:
model_complex = ConvNet_complex(num_classes=5, dropout_prob=0.1).to(device)
optimizer_complex=Adam(model_complex.parameters(),lr=0.001,weight_decay=0.0001)

complex ReLu max pooling batch size 100, no class weights learning rate = 0.001

In [ ]:
# Model training and saving the best model

# Initialize the best accuracy variable
best_accuracy = 0.0

# Iterate through each epoch
for epoch in range(num_epochs):
    
    # Set the model to training mode
    model_complex.train()
    
    # Initialize training accuracy and loss
    train_accuracy = 0.0
    train_loss = 0.0
    
    # Iterate through the training dataset
    for i, (images, labels) in enumerate(train_loader_100):
        # Move images and labels to GPU if available
        if torch.cuda.is_available():
            images = Variable(images.cuda())
            labels = Variable(labels.cuda())
            
        # Zero the gradients
        optimizer_complex.zero_grad()
        
        # Forward pass
        outputs = model_complex(images)
        
        # Compute the loss
        loss = loss_function(outputs, labels)
        
        # Backward pass
        loss.backward()
        
        # Update the weights
        optimizer_complex.step()
        
        # Update training loss
        train_loss += loss.cpu().data * images.size(0)
        
        # Calculate training accuracy
        _, prediction = torch.max(outputs.data, 1)
        train_accuracy += int(torch.sum(prediction == labels.data))
        
    # Calculate average training accuracy and loss
    train_accuracy = train_accuracy / train_count
    train_loss = train_loss / train_count
    # Initialize the confusion matrix
    conf_matrix = torch.zeros(5, 5)

    # Set the model to evaluation mode
    model_complex.eval()
    
    # Initialize testing accuracy
    test_accuracy = 0.0
    
    # Iterate through the testing dataset
    for i, (images, labels) in enumerate(test_loader_100):
        # Move images and labels to GPU if available
        if torch.cuda.is_available():
            images = Variable(images.cuda())
            labels = Variable(labels.cuda())
            
        # Forward pass
        outputs = model_complex(images)
        
        # Calculate testing accuracy
        _, prediction = torch.max(outputs.data, 1)
        test_accuracy += int(torch.sum(prediction == labels.data))
        # Update the confusion matrix
        conf_matrix += confusion_matrix(labels.cpu().numpy(), prediction.cpu().numpy(), labels=list(range(5)))
    
    # Calculate average testing accuracy
    test_accuracy = test_accuracy / test_count
    class_accuracy = conf_matrix.diag() / conf_matrix.sum(1)

    # Print class-wise accuracy
    for i in range(5):
        print(f'Class {i} Accuracy: {class_accuracy[i].item()}')
        # scheduler.step()
        # Print epoch-wise results
    print('Epoch: ' + str(epoch) + ' Train Loss: ' + str(train_loss) +
        ' Train Accuracy: ' + str(train_accuracy) + ' Test Accuracy: ' + str(test_accuracy))
    
    # Save the best model based on testing accuracy
    if test_accuracy > best_accuracy:
        torch.save(model_complex.state_dict(), 'best_checkpoint_dropout.model')
        best_accuracy = test_accuracy



In [ ]:
#Dataloader

#Path for training and testing directory
train_path='./gray_data/train'
test_path='./gray_data/test'

#set the batch size for CNN
train_loader_32=DataLoader(
    torchvision.datasets.ImageFolder(train_path,transform=transformer),
    batch_size=32, shuffle=True
)
test_loader_32=DataLoader(
    torchvision.datasets.ImageFolder(test_path,transform=transformer),
    batch_size=32, shuffle=True
)

In [ ]:
model_dropout = ConvNet_dropout(num_classes=5, dropout_prob=0.2).to(device)
optimizer_dropout=Adam(model_dropout.parameters(),lr=0.001,weight_decay=0.0001)

grey 

In [ ]:
# Model training and saving the best model

# Initialize the best accuracy variable
best_accuracy = 0.0

# Iterate through each epoch
for epoch in range(num_epochs):
    
    # Set the model to training mode
    model_dropout.train()
    
    # Initialize training accuracy and loss
    train_accuracy = 0.0
    train_loss = 0.0
    
    # Iterate through the training dataset
    for i, (images, labels) in enumerate(train_loader_32):
        # Move images and labels to GPU if available
        if torch.cuda.is_available():
            images = Variable(images.cuda())
            labels = Variable(labels.cuda())
            
        # Zero the gradients
        optimizer_dropout.zero_grad()
        
        # Forward pass
        outputs = model_dropout(images)
        
        # Compute the loss
        loss = loss_function(outputs, labels)
        
        # Backward pass
        loss.backward()
        
        # Update the weights
        optimizer_dropout.step()
        
        # Update training loss
        train_loss += loss.cpu().data * images.size(0)
        
        # Calculate training accuracy
        _, prediction = torch.max(outputs.data, 1)
        train_accuracy += int(torch.sum(prediction == labels.data))
        
    # Calculate average training accuracy and loss
    train_accuracy = train_accuracy / train_count
    train_loss = train_loss / train_count
    # Initialize the confusion matrix
    conf_matrix = torch.zeros(5, 5)

    # Set the model to evaluation mode
    model_dropout.eval()
    
    # Initialize testing accuracy
    test_accuracy = 0.0
    
    # Iterate through the testing dataset
    for i, (images, labels) in enumerate(test_loader_32):
        # Move images and labels to GPU if available
        if torch.cuda.is_available():
            images = Variable(images.cuda())
            labels = Variable(labels.cuda())
            
        # Forward pass
        outputs = model_dropout(images)
        
        # Calculate testing accuracy
        _, prediction = torch.max(outputs.data, 1)
        test_accuracy += int(torch.sum(prediction == labels.data))
        # Update the confusion matrix
        conf_matrix += confusion_matrix(labels.cpu().numpy(), prediction.cpu().numpy(), labels=list(range(5)))
    
    # Calculate average testing accuracy
    test_accuracy = test_accuracy / test_count
    class_accuracy = conf_matrix.diag() / conf_matrix.sum(1)

    # Print class-wise accuracy
    for i in range(5):
        print(f'Class {i} Accuracy: {class_accuracy[i].item()}')
        # scheduler.step()
        # Print epoch-wise results
    print('Epoch: ' + str(epoch) + ' Train Loss: ' + str(train_loss) +
        ' Train Accuracy: ' + str(train_accuracy) + ' Test Accuracy: ' + str(test_accuracy))
    
    # Save the best model based on testing accuracy
    if test_accuracy > best_accuracy:
        torch.save(model_dropout.state_dict(), 'best_checkpoint_dropout.model')
        best_accuracy = test_accuracy



In [ ]:
#Dataloader

#Path for training and testing directory
train_path='./sharpened_data/train'
test_path='./sharpened_data/test'

#set the batch size for CNN
train_loader_32=DataLoader(
    torchvision.datasets.ImageFolder(train_path,transform=transformer),
    batch_size=32, shuffle=True
)
test_loader_32=DataLoader(
    torchvision.datasets.ImageFolder(test_path,transform=transformer),
    batch_size=32, shuffle=True
)

In [ ]:
model_dropout = ConvNet_dropout(num_classes=5, dropout_prob=0.2).to(device)
optimizer_dropout=Adam(model_dropout.parameters(),lr=0.001,weight_decay=0.0001)

sharpen

In [ ]:
# Model training and saving the best model

# Initialize the best accuracy variable
best_accuracy = 0.0

# Iterate through each epoch
for epoch in range(num_epochs):
    
    # Set the model to training mode
    model_dropout.train()
    
    # Initialize training accuracy and loss
    train_accuracy = 0.0
    train_loss = 0.0
    
    # Iterate through the training dataset
    for i, (images, labels) in enumerate(train_loader_32):
        # Move images and labels to GPU if available
        if torch.cuda.is_available():
            images = Variable(images.cuda())
            labels = Variable(labels.cuda())
            
        # Zero the gradients
        optimizer_dropout.zero_grad()
        
        # Forward pass
        outputs = model_dropout(images)
        
        # Compute the loss
        loss = loss_function(outputs, labels)
        
        # Backward pass
        loss.backward()
        
        # Update the weights
        optimizer_dropout.step()
        
        # Update training loss
        train_loss += loss.cpu().data * images.size(0)
        
        # Calculate training accuracy
        _, prediction = torch.max(outputs.data, 1)
        train_accuracy += int(torch.sum(prediction == labels.data))
        
    # Calculate average training accuracy and loss
    train_accuracy = train_accuracy / train_count
    train_loss = train_loss / train_count
    # Initialize the confusion matrix
    conf_matrix = torch.zeros(5, 5)

    # Set the model to evaluation mode
    model_dropout.eval()
    
    # Initialize testing accuracy
    test_accuracy = 0.0
    
    # Iterate through the testing dataset
    for i, (images, labels) in enumerate(test_loader_32):
        # Move images and labels to GPU if available
        if torch.cuda.is_available():
            images = Variable(images.cuda())
            labels = Variable(labels.cuda())
            
        # Forward pass
        outputs = model_dropout(images)
        
        # Calculate testing accuracy
        _, prediction = torch.max(outputs.data, 1)
        test_accuracy += int(torch.sum(prediction == labels.data))
        # Update the confusion matrix
        conf_matrix += confusion_matrix(labels.cpu().numpy(), prediction.cpu().numpy(), labels=list(range(5)))
    
    # Calculate average testing accuracy
    test_accuracy = test_accuracy / test_count
    class_accuracy = conf_matrix.diag() / conf_matrix.sum(1)

    # Print class-wise accuracy
    for i in range(5):
        print(f'Class {i} Accuracy: {class_accuracy[i].item()}')
        # scheduler.step()
        # Print epoch-wise results
    print('Epoch: ' + str(epoch) + ' Train Loss: ' + str(train_loss) +
        ' Train Accuracy: ' + str(train_accuracy) + ' Test Accuracy: ' + str(test_accuracy))
    
    # Save the best model based on testing accuracy
    if test_accuracy > best_accuracy:
        torch.save(model_dropout.state_dict(), 'best_checkpoint_dropout.model')
        best_accuracy = test_accuracy



In [ ]:
#Dataloader

#Path for training and testing directory
train_path='./resized_data_split_less/train'
test_path='./resized_data_split_less/test'

#set the batch size for CNN
train_loader_32=DataLoader(
    torchvision.datasets.ImageFolder(train_path,transform=transformer),
    batch_size=32, shuffle=True
)
test_loader_32=DataLoader(
    torchvision.datasets.ImageFolder(test_path,transform=transformer),
    batch_size=32, shuffle=True
)

In [ ]:
model_dropout = ConvNet_dropout(num_classes=4, dropout_prob=0.2).to(device)
optimizer_dropout=Adam(model_dropout.parameters(),lr=0.001,weight_decay=0.0001)

without class 3

In [ ]:
# Model training and saving the best model

# Initialize the best accuracy variable
best_accuracy = 0.0

# Iterate through each epoch
for epoch in range(num_epochs):
    
    # Set the model to training mode
    model_dropout.train()
    
    # Initialize training accuracy and loss
    train_accuracy = 0.0
    train_loss = 0.0
    
    # Iterate through the training dataset
    for i, (images, labels) in enumerate(train_loader_32):
        # Move images and labels to GPU if available
        if torch.cuda.is_available():
            images = Variable(images.cuda())
            labels = Variable(labels.cuda())
            
        # Zero the gradients
        optimizer_dropout.zero_grad()
        
        # Forward pass
        outputs = model_dropout(images)
        
        # Compute the loss
        loss = loss_function(outputs, labels)
        
        # Backward pass
        loss.backward()
        
        # Update the weights
        optimizer_dropout.step()
        
        # Update training loss
        train_loss += loss.cpu().data * images.size(0)
        
        # Calculate training accuracy
        _, prediction = torch.max(outputs.data, 1)
        train_accuracy += int(torch.sum(prediction == labels.data))
        
    # Calculate average training accuracy and loss
    train_accuracy = train_accuracy / train_count
    train_loss = train_loss / train_count
    # Initialize the confusion matrix
    conf_matrix = torch.zeros(5, 5)

    # Set the model to evaluation mode
    model_dropout.eval()
    
    # Initialize testing accuracy
    test_accuracy = 0.0
    
    # Iterate through the testing dataset
    for i, (images, labels) in enumerate(test_loader_32):
        # Move images and labels to GPU if available
        if torch.cuda.is_available():
            images = Variable(images.cuda())
            labels = Variable(labels.cuda())
            
        # Forward pass
        outputs = model_dropout(images)
        
        # Calculate testing accuracy
        _, prediction = torch.max(outputs.data, 1)
        test_accuracy += int(torch.sum(prediction == labels.data))
        # Update the confusion matrix
        conf_matrix += confusion_matrix(labels.cpu().numpy(), prediction.cpu().numpy(), labels=list(range(5)))
    
    # Calculate average testing accuracy
    test_accuracy = test_accuracy / test_count
    class_accuracy = conf_matrix.diag() / conf_matrix.sum(1)

    # Print class-wise accuracy
    for i in range(4):
        print(f'Class {i} Accuracy: {class_accuracy[i].item()}')
        # scheduler.step()
        # Print epoch-wise results
    print('Epoch: ' + str(epoch) + ' Train Loss: ' + str(train_loss) +
        ' Train Accuracy: ' + str(train_accuracy) + ' Test Accuracy: ' + str(test_accuracy))
    
    # Save the best model based on testing accuracy
    if test_accuracy > best_accuracy:
        torch.save(model_dropout.state_dict(), 'best_checkpoint_dropout.model')
        best_accuracy = test_accuracy

